# Manual classification

Manual classificato to set the labels for training and ML classification

In [ ]:
using PyCall
using DataFrames, Query

using Random , Printf
using Images , ImageShow,  PNGFiles , ImageView

rootdir = ENV["GAIA_ROOT"]

push!(LOAD_PATH,"$rootdir/run/src")
using GaiaClustering

import PyPlot , CSV
mpimg= PyCall.pyimport("matplotlib.image")

wdir    = "$rootdir/products"
ocplot  = "/home/stephane/Science/GAIA/e2e_products.edr3/res.alma.edr3/e2e_products.edr3/plotSelect"

In [ ]:
ocsc= "ocres-test.sc.csv"
oc_class= "ocres-test.class.csv"

df= CSV.read(joinpath(wdir,ocsc), delim= ";", DataFrame)

dfclass= DataFrame()
isclass= false
## if already started, read it here:
##
if isfile(joinpath(wdir,oc_class))
    dfclass= CSV.read(joinpath(wdir,oc_class), delim= ";", DataFrame)
    isclass= true
end

println("## instructions")
println("## n: not detected (false)")
println("## u: unknown")
println("## c: cluster detected")
println("## x: exit and save")

ichecked= 0
for oc in shuffle(eachrow(df))
    
    if isclass
        df_filter= @from i in dfclass begin
            @where i.votname == oc.votname && i.cycle == oc.cycle
            @select {i.votname}
            @collect DataFrame
        end
    end
    
    if isclass
        sclass= size(df_filter, 1)
    else
        sclass= 0
    end
    
    if sclass == 0 
    
        clfile= @sprintf("%s.%d.cluster.png",oc.votname,oc.cycle)
        clim= joinpath(ocplot,clfile)
        
        rawfile= @sprintf("%s.%d.raw.png",oc.votname,oc.cycle)
        rawim= joinpath(ocplot,rawfile)
        
        img1= PNGFiles.load(clim)
        img2= PNGFiles.load(rawim)
        
        ImageView.imshow(img1)
        ImageView.imshow(img2)

        
        ans= false
        listAns = ["n", "u", "c"]
        rep= readline()
    
        if rep == "x"
            println("## Save and exit...")
            CSV.write(joinpath(wdir,oc_class), dfclass, delim= ";")     
            break
        end
    
        ichecked += 1 
        @printf("### %d checked \n", ichecked)
        dft= DataFrame(votname= oc.votname, cycle= oc.cycle, class= rep)
        append!(dfclass, dft)
        isclass= true
    end
end